In [ ]:
%pip install tensorflow matplotlib scikit-learn pydot

In [ ]:
from tensorflow import keras
from keras import backend
from keras.callbacks import ModelCheckpoint
from keras.layers import Activation, Add, Dense, Input, Lambda
from keras.models import Model
import numpy as np
import requests

In [ ]:
INPUT_DIM = 24
h_1_dim = 64
h_2_dim = h_1_dim // 2
h_3_dim = h_2_dim // 2

# Model.
h_1 = Dense(h_1_dim, activation = "relu")
h_2 = Dense(h_2_dim, activation = "relu")
h_3 = Dense(h_3_dim, activation = "relu")
s = Dense(1)

# Relevant document score.
rel_doc = Input(shape = (INPUT_DIM, ), dtype = "float32")
h_1_rel = h_1(rel_doc)
h_2_rel = h_2(h_1_rel)
h_3_rel = h_3(h_2_rel)
rel_score = s(h_3_rel)

# Irrelevant document score.
irr_doc = Input(shape = (INPUT_DIM, ), dtype = "float32")
h_1_irr = h_1(irr_doc)
h_2_irr = h_2(h_1_irr)
h_3_irr = h_3(h_2_irr)
irr_score = s(h_3_irr)

# Subtract scores.
negated_irr_score = Lambda(lambda x: -1 * x, output_shape = (1, ))(irr_score)
diff = Add()([rel_score, negated_irr_score])

# Pass difference through sigmoid function.
prob = Activation("sigmoid")(diff)

# Build model.
model = Model(inputs = [rel_doc, irr_doc], outputs = prob)
model.compile(optimizer = "adagrad", loss = "binary_crossentropy")

In [ ]:
from keras.utils.vis_utils import plot_model

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
from os import listdir
from os.path import isfile, join
def getJudgmentsBatchFileByFile():
    """
    Returns a generator function that returns all the judgment batches files from the directory
    """
    files = []
   
    files = [join('./loggedFeatures', file) for file in listdir('./loggedFeatures') if isfile(join('./loggedFeatures', file))]
    for file in files:
        yield file

In [ ]:
import csv
judgments = []
for file in getJudgmentsBatchFileByFile():
  with open(file,'r') as f:
    reader = csv.reader(f, delimiter=' ')
    for row in reader:
      data = []
      for element in row:
        data.append(element.replace(',', ''))
      judgments.append(data)

In [ ]:
judgments.sort(key = lambda judgments: judgments[1])
print(judgments[0])
print(judgments[1])
print(judgments[2])
print(judgments[3])

In [ ]:
for j in judgments:
  del j[2]

print(judgments[0])
print(judgments[1])
print(judgments[2])
print(judgments[3])
print(judgments[4])
print(judgments[5])
print(judgments[6])
print(judgments[7])

In [ ]:
#create pairs
xi = []
xj = []
pij = []
judgmentPairs = []

for i in range(1, len(judgments), 2):
  judgmentPairs.append([judgments[i - 1], judgments[i]])
  xi.append(judgments[i - 1][2:])
  xj.append(judgments[i][2:])
  if judgments[i-1][0] == judgments[i][0]:
    _pij = 0.5
  elif judgments[i-1][0] > judgments[i][0]:
    _pij = 1
  else: 
    _pij = 0
  pij.append(_pij)

In [ ]:
xi = np.array(xi, dtype='float64')

In [ ]:
xj = np.array(xj, dtype='float64')

In [ ]:
pij = np.array(pij, dtype='float64')

In [ ]:
from sklearn.model_selection import train_test_split

xi_train, xi_test, xj_train, xj_test, pij_train, pij_test = train_test_split(
    xi, xj, pij, test_size=0.2, shuffle=False)

In [ ]:
NUM_EPOCHS = 30
BATCH_SIZE = 32
checkpointer = ModelCheckpoint(filepath = "training/valid_params.h5", verbose = 1, save_best_only = True)
history = model.fit([xi_train, xj_train], pij_train,
                     epochs = NUM_EPOCHS, batch_size = BATCH_SIZE, validation_data=([xi_test, xj_test], pij_test),
                     callbacks = [checkpointer])

In [ ]:
import json
modelName = "thesis-ranknet"
solrModel = {"store" : "thesis-ltr",
              "name" :  modelName,
              "class" : "org.apache.solr.ltr.model.NeuralNetworkModel",
              "features" : [
                { "name" : "title_coveredQueryTerms" },
                { "name" : "headings_coveredQueryTerms" },
                { "name" : "body_coveredQueryTerms" },
                { "name" : "document_coveredQueryTerms" },
                { "name" : "title_coveredQueryTermsRatio" },
                { "name" : "headings_coveredQueryTermsRatio" },
                { "name" : "body_coveredQueryTermsRatio" },
                { "name" : "document_coveredQueryTermsRatio" },
                { "name" : "title_tf" },
                { "name" : "headings_tf" },
                { "name" : "body_tf" },
                { "name" : "document_tf" },
                { "name" : "title_idf" },
                { "name" : "headings_idf" },
                { "name" : "body_idf" },
                { "name" : "document_idf" },
                { "name" : "title_tfidf" },
                { "name" : "headings_tfidf" },
                { "name" : "body_tfidf" },
                { "name" : "document_tfidf" },
                { "name" : "title_bm25" },
                { "name" : "headings_bm25" },
                { "name" : "body_bm25" },
                { "name" : "document_bm25" }
              ],
              "params": {}}

weights = model.get_weights()

layers = []
layers.append({"matrix": weights[0].T.tolist(),
               "bias": weights[1].tolist(),
               "activation": "relu"})
layers.append({"matrix": weights[2].T.tolist(),
               "bias": weights[3].tolist(),
               "activation": "relu"})
layers.append({"matrix": weights[4].T.tolist(),
              "bias": weights[5].tolist(),
              "activation": "relu"})
layers.append({"matrix": weights[6].T.tolist(),
              "bias": weights[7].tolist(),
              "activation": "identity"})
solrModel["params"]["layers"] = layers


print(json.dumps(solrModel, indent=2))

# Upload the model after deleting the model
print('Delete')
requests.delete(f'http://localhost:8983/solr/thesis-ltr/schema/model-store/{modelName}').json()
print('Add new model')
requests.put(f'http://localhost:8983/solr/thesis-ltr/schema/model-store', json=solrModel).json()